ADVANCE LEARNING ALGORITHMS 

WEEK 2

ASSIGNMENT

In [26]:
# THESE ARE THE LIBRARIES NEEDED TO IMPLEMENT THE MODEL IN PYTORCH
import torch
import torch.nn as nn
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset

IMPORTING MNIST DATASET

In [27]:
# I HAVE DOWNLOADED THE DATASET FORM SKLEARN LIBRARY. MINIST DATASET IS AN IMAGE DATASET OF HANDWRITTEN DIGITS.
# THE DATASET CONTAINS 70,000 IMAGES OF HANDWRITTEN DIGITS. EACH IMAGE IS 28X28 PIXELS.

from sklearn.datasets import fetch_openml
mnist = fetch_openml('mnist_784', version=1)

X, y = mnist['data'], mnist['target']


from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

# NORMALIZE THE DATA
# THIS IS DONE TO MAKE THE TRAINING PROCESS FASTER
# THE PIXEL VALUES OF THE IMAGES ARE IN THE RANGE OF 0 TO 255
# WE DIVIDE THE PIXEL VALUES BY 255 SO THAT THE PIXEL VALUES WILL BE IN THE RANGE OF 0 TO 1

x_train = x_train / 255.0
x_test = x_test / 255.0


print(f"x_train shape: {x_train.shape}")
print(f"y_train shape: {y_train.shape}")
print(f"x_test shape: {x_test.shape}")
print(f"y_test shape: {y_test.shape}")


x_train shape: (52500, 784)
y_train shape: (52500,)
x_test shape: (17500, 784)
y_test shape: (17500,)


Creating pytorch tensors

In [28]:
# THE DATA IS IN THE FORM OF PANDAS DATAFRAME
# WE NEED TO CONVERT THE DATA INTO TENSORS TO USE IT IN PYTORCH
# BELOW CODE CONVERTS THE DATA INTO TENSORS

X = torch.tensor(x_train.values, dtype=torch.float32)
x = torch.tensor(x_test.values, dtype=torch.float32)
Y = torch.tensor(y_train.values.astype(int), dtype=torch.long)
y = torch.tensor(y_test.values.astype(int), dtype=torch.long)

Creating the desired NN model for the dataset

In [29]:

# CHECK GPU, IF AVAILABLE, USE IT
# I HAVE MADE SURE THAT THE TENSORS AND THE MODEL IS ALSO SENT TO THE GPU FOR TRAINING IN THE CODE BELOW 
# .to(device) SENDS THE DATA TO THE GPU

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
batch_size = 32
train_dataset = TensorDataset(X,Y)
test_dataset = TensorDataset(x, y)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)


# DEFINING THE DESIRED NN MODEL IN PYTORCH 
# THE MODEL CONTAINS 3 LAYERS
# THE INPUT LAYER HAS 784 NEURONS
# THE HIDDEN LAYER HAS 128 NEURONS
# THE HIDDEN LAYER HAS 64 NEURONS
# THE OUTPUT LAYER HAS 10 NEURONS
# THE OUTPUT LAYER HAS 10 NEURONS BECAUSE THERE ARE 10 CLASSES IN THE DATASET

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(784, 128)
        self.fc2 = nn.Linear(128, 64)
        self.fc3 = nn.Linear(64, 10)
    
    # THIS IS FORWARD PASS OF NEURAL NETWORK
    # THE INPUT IS FLATTENED TO 1D TENSOR
    # WE HAVE USED RELU ACTIVATION FUNCTION IN THE HIDDEN LAYERS
    def forward(self, x):
        x = torch.flatten(x, start_dim=1)
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = self.fc3(x)
        return x



# HERE WE ENSURE THE MODEL IS SENT TO THE GPU.
# THE DEVICE VARIABLE CONTAINS THE INFORMATION ABOUT THE GPU
# IF IT IS AVALABLE, THE MODEL WILL BE SENT TO THE GPU
# ELSE IT WILL BE SENT TO THE CPU

model = Net().to(device)

# THE LOSS FUNCTION USED IS CROSS ENTROPY LOSS
# THE OPTIMIZER USED IS ADAM OPTIMIZER
# THE LEARNING RATE IS SET TO BE 0.001

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)


# TRAINING THE MODEL
# THE MODEL IS TRAINED FOR 10 EPOCHS
# THE TRAINING LOSS IS PRINTED AFTER EACH EPOCH
# THE TRAINING LOSS DECREASES AS THE EPOCHS INCREASES
epochs = 10
for epoch in range(epochs):
    running_loss = 0.0
    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device) # SEND DATA TO GPU
        optimizer.zero_grad()
        outputs = model(inputs) # FORWARD PASS. RESULTS ARE STORED IN OUTPUTS
        loss = criterion(outputs, labels) # CALCULATE LOSS
        loss.backward() # BACKWARD PASS
        optimizer.step() # UPDATE WEIGHTS
        running_loss += loss.item() # ADD LOSS TO RUNNING LOSS.
    print(f"Epoch {epoch + 1}/{epochs}, Loss: {running_loss / len(train_loader)}")


# TESTING THE MODEL
# HERE WE PRINT THE ACCURACY OF THE MODEL
# THE MODEL IS TESTED ON THE TEST SET
# THE ACCURACY OF THE MODEL IS PRINTED

correct = 0
total = 0
with torch.no_grad():
    for inputs, labels in test_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = model(inputs)
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'Accuracy on test set: {100 * correct / total}%')


Epoch 1/10, Loss: 0.31009370770128325
Epoch 2/10, Loss: 0.13009191055979075
Epoch 3/10, Loss: 0.08875649674604877
Epoch 4/10, Loss: 0.06663623656477384
Epoch 5/10, Loss: 0.05134473633925481
Epoch 6/10, Loss: 0.039817361865439674
Epoch 7/10, Loss: 0.0331788550013375
Epoch 8/10, Loss: 0.028503139206508956
Epoch 9/10, Loss: 0.025275266671198438
Epoch 10/10, Loss: 0.02032229780665417
Accuracy on test set: 97.48571428571428%
